# 🧪 AI Model Playground

**Test any HuggingFace model in minutes.** Just change `MODEL_ID` and run the cells.

---

## 1. Install Dependencies

In [ ]:
# Run once – installs everything you need
%pip install torch torchvision transformers huggingface-hub accelerate timm peft pillow sentencepiece safetensors requests rich click matplotlib pandas -q

## 2. Import Libraries

In [ ]:
import torch
import requests
import os
import sys
import json
import importlib.util
from pathlib import Path
from io import BytesIO

from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd

from transformers import (
    AutoConfig,
    AutoModel,
    AutoModelForImageClassification,
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    AutoProcessor,
    AutoImageProcessor,
    AutoFeatureExtractor,
    pipeline,
)
from huggingface_hub import HfApi, hf_hub_download, snapshot_download

print(f"PyTorch: {torch.__version__}")
print(f"Device:  {'MPS' if torch.backends.mps.is_available() else 'CUDA' if torch.cuda.is_available() else 'CPU'}")

## 3. Configure Environment & Authentication

Set your HuggingFace token if the model is gated (e.g. Llama, Gemma).  
For public models you can skip this.

In [ ]:
# ── Device setup ──────────────────────────────────────────────────────────────
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
elif torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")
print(f"Using device: {DEVICE}")

# ── HuggingFace auth (uncomment if you need gated models) ────────────────────
# from huggingface_hub import login
# login()  # will prompt for your token

# Or set it directly:
# os.environ["HF_TOKEN"] = "hf_your_token_here"

# ── Local cache dir ──────────────────────────────────────────────────────────
MODELS_DIR = Path("models")
MODELS_DIR.mkdir(exist_ok=True)

## 4. Model Loading Utility

A universal loader that auto-detects model type and returns the right model + processor.

In [ ]:
# Map HuggingFace pipeline_tag → best Auto class
TASK_TO_MODEL_CLASS = {
    "image-classification":      AutoModelForImageClassification,
    "text-classification":       AutoModelForSequenceClassification,
    "sentiment-analysis":        AutoModelForSequenceClassification,
    "text-generation":           AutoModelForCausalLM,
    "text2text-generation":      AutoModelForSeq2SeqLM,
    "feature-extraction":        AutoModel,
}

def get_model_info(model_id: str) -> dict:
    """Fetch metadata from HuggingFace Hub."""
    api = HfApi()
    info = api.model_info(model_id)
    files = [s.rfilename for s in (info.siblings or [])]
    return {
        "model_id":     model_id,
        "pipeline_tag": info.pipeline_tag or "",
        "library":      info.library_name or "",
        "tags":         info.tags or [],
        "files":        files,
    }

def download_repo(model_id: str) -> Path:
    """Download full repo into models/ folder."""
    safe = model_id.replace("/", "__")
    local_dir = MODELS_DIR / safe
    if local_dir.exists() and any(local_dir.iterdir()):
        print(f"✓ Already downloaded: {local_dir}")
        return local_dir
    print(f"↓ Downloading {model_id} …")
    snapshot_download(repo_id=model_id, local_dir=str(local_dir),
                      ignore_patterns=["*.md", ".gitattributes"])
    print(f"✓ Done → {local_dir}")
    return local_dir

def load_any_model(model_id: str, device=DEVICE):
    """
    Universal loader. Returns (model_or_module, processor_or_None, info).

    - Standard HF models → returns (model, processor, info)
    - Custom models (has model.py) → downloads repo, imports module, returns (module, None, info)
    - Pipeline fallback → returns (pipeline, None, info)
    """
    info = get_model_info(model_id)
    task = info["pipeline_tag"]
    files = info["files"]
    print(f"Model:  {model_id}")
    print(f"Task:   {task or 'unknown'}")
    print(f"Library: {info['library'] or 'unknown'}")
    print(f"Files:  {len(files)}")

    # ── Custom code models (model.py in repo) ────────────────────────────
    has_custom = any(f in files for f in ["model.py", "inference.py", "predict.py"])
    if has_custom:
        local_dir = download_repo(model_id)
        custom_file = next(f for f in ["model.py", "inference.py", "predict.py"] if f in files)
        # Add to path so imports work
        if str(local_dir) not in sys.path:
            sys.path.insert(0, str(local_dir))
        spec = importlib.util.spec_from_file_location("custom_model", local_dir / custom_file)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)
        exports = [n for n in dir(module) if not n.startswith("_") and callable(getattr(module, n))]
        print(f"\n✓ Custom module loaded from {custom_file}")
        print(f"  Available: {', '.join(exports)}")
        return module, None, info

    # ── Standard HF pipeline model ───────────────────────────────────────
    model_cls = TASK_TO_MODEL_CLASS.get(task)

    if model_cls:
        print(f"⏳ Loading with {model_cls.__name__} …")
        model = model_cls.from_pretrained(model_id).to(device).eval()

        # Try loading processor / tokenizer
        processor = None
        for loader in [AutoProcessor, AutoImageProcessor, AutoTokenizer, AutoFeatureExtractor]:
            try:
                processor = loader.from_pretrained(model_id)
                break
            except Exception:
                continue

        n_params = sum(p.numel() for p in model.parameters())
        print(f"✓ Loaded — {n_params/1e6:.1f}M parameters on {device}")

        # Show labels if available
        if hasattr(model.config, "id2label") and model.config.id2label:
            labels = model.config.id2label
            print(f"  Labels ({len(labels)}): {dict(list(labels.items())[:5])} …")

        return model, processor, info

    # ── Fallback: HF pipeline API ────────────────────────────────────────
    if task:
        print(f"⏳ Loading via pipeline('{task}') …")
        pipe = pipeline(task, model=model_id, device=device if str(device) != "mps" else -1)
        print(f"✓ Pipeline ready")
        return pipe, None, info

    # ── Last resort: download and let user figure it out ─────────────────
    local_dir = download_repo(model_id)
    print(f"⚠ Unknown task type. Files downloaded to: {local_dir}")
    return None, None, info

print("✓ Utility ready")

## 5. Load a Model

**Change `MODEL_ID` to test any model.** Just paste a HuggingFace URL or `org/model` string.

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  CHANGE THIS to test any model                                             ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

MODEL_ID = "Bombek1/ai-image-detector-siglip-dinov2"

# More examples – uncomment any line:
# MODEL_ID = "google/vit-base-patch16-224"              # image classification
# MODEL_ID = "microsoft/resnet-50"                       # image classification
# MODEL_ID = "facebook/detr-resnet-50"                   # object detection
# MODEL_ID = "distilbert-base-uncased-finetuned-sst-2-english"  # sentiment
# MODEL_ID = "google/gemma-2-2b-it"                     # text generation (needs HF token)
# MODEL_ID = "microsoft/Phi-3-mini-4k-instruct"         # text generation
# MODEL_ID = "openai/whisper-base"                       # speech recognition

model, processor, info = load_any_model(MODEL_ID)

## 6. Prepare Sample Input

Helper functions for loading images, text, or audio depending on the model type.

In [ ]:
def load_image(source: str) -> Image.Image:
    """Load an image from a URL or local file path."""
    if source.startswith("http"):
        response = requests.get(source, timeout=15)
        img = Image.open(BytesIO(response.content)).convert("RGB")
    else:
        img = Image.open(source).convert("RGB")
    return img

def show_image(img: Image.Image, title: str = ""):
    """Display an image inline."""
    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    plt.axis("off")
    if title:
        plt.title(title, fontsize=14)
    plt.tight_layout()
    plt.show()

# ── Load a sample image ─────────────────────────────────────────────────────
SAMPLE_IMAGE_URL = "https://images.unsplash.com/photo-1574158622682-e40e69881006?w=400"  # cat photo

sample_image = load_image(SAMPLE_IMAGE_URL)
show_image(sample_image, "Sample Input Image")

## 7. Run Inference

This section handles both **standard HF models** and **custom models** (like the AI image detector).

In [ ]:
task = info.get("pipeline_tag", "")
has_custom = any(f in info.get("files", []) for f in ["model.py", "inference.py", "predict.py"])

if has_custom:
    # ── Custom model (e.g. Bombek1/ai-image-detector-siglip-dinov2) ──────
    print(f"Custom model detected — using module from model.py")
    print(f"Available: {[n for n in dir(model) if not n.startswith('_') and callable(getattr(model, n))]}")

    # For the AI image detector specifically:
    if hasattr(model, "AIImageDetector"):
        model_path = hf_hub_download(
            repo_id=MODEL_ID,
            filename="pytorch_model.pt"
        )
        detector = model.AIImageDetector(model_path)
        result = detector.predict(SAMPLE_IMAGE_URL)
        print(f"\nPrediction: {result['prediction']}")
        print(f"Confidence: {result['confidence']:.1%}")
        print(f"P(AI):      {result['probability']:.4f}")
    else:
        print("\n⚠ Check the available classes/functions above and call them manually.")
        print("  Example: result = model.SomeClass(args).predict(input)")

elif task in ("image-classification", "object-detection", "image-segmentation"):
    # ── Vision models ────────────────────────────────────────────────────
    if processor is not None:
        inputs = processor(images=sample_image, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=-1)[0]
        top5 = probs.topk(5)
        results = []
        for score, idx in zip(top5.values, top5.indices):
            label = model.config.id2label.get(idx.item(), f"class_{idx.item()}")
            results.append({"Label": label, "Confidence": f"{score.item():.4f}"})
        df = pd.DataFrame(results)
        print(df.to_string(index=False))
    else:
        # Pipeline fallback
        pipe = pipeline(task, model=MODEL_ID)
        results = pipe(sample_image)
        for r in results[:5]:
            print(f"  {r['label']}: {r['score']:.4f}")

elif task in ("text-generation", "text2text-generation"):
    # ── Text generation ──────────────────────────────────────────────────
    prompt = "Explain quantum computing in one paragraph."
    if processor:
        inputs = processor(prompt, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            out = model.generate(**inputs, max_new_tokens=200, temperature=0.7, do_sample=True)
        print(processor.decode(out[0], skip_special_tokens=True))
    else:
        result = model(prompt, max_new_tokens=200)
        print(result[0]["generated_text"])

elif task in ("text-classification", "sentiment-analysis"):
    # ── Text classification ──────────────────────────────────────────────
    text = "I absolutely love this product, it changed my life!"
    if processor:
        inputs = processor(text, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)[0]
        for idx, score in enumerate(probs):
            label = model.config.id2label.get(idx, f"class_{idx}")
            print(f"  {label}: {score.item():.4f}")
    else:
        result = model(text)
        for r in result:
            print(f"  {r['label']}: {r['score']:.4f}")

else:
    print(f"Task '{task}' — use the model object directly or try:")
    print(f"  pipe = pipeline('{task}', model='{MODEL_ID}')")
    print(f"  result = pipe(your_input)")

## 8. Display & Visualise Results

For vision models: show the image with prediction overlay.  
For text models: results are already printed above.

In [ ]:
# If we got tabular results from a vision model, show as a bar chart
if task in ("image-classification",) and not has_custom and 'df' in dir():
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    axes[0].imshow(sample_image)
    axes[0].axis("off")
    axes[0].set_title("Input Image")

    df_plot = df.copy()
    df_plot["Confidence"] = df_plot["Confidence"].astype(float)
    axes[1].barh(df_plot["Label"], df_plot["Confidence"], color="steelblue")
    axes[1].set_xlabel("Confidence")
    axes[1].set_title("Top-5 Predictions")
    axes[1].invert_yaxis()
    plt.tight_layout()
    plt.show()
elif has_custom and 'result' in dir() and isinstance(result, dict):
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(sample_image)
    ax.axis("off")
    pred = result.get("prediction", "?")
    conf = result.get("confidence", 0)
    color = "red" if "ai" in str(pred).lower() else "green"
    ax.set_title(f"{pred} ({conf:.1%})", fontsize=16, color=color)
    plt.tight_layout()
    plt.show()
else:
    print("ℹ Results displayed in the cell above.")

## 9. Test with Your Own Input

Paste your own image URL, file path, or text below.

In [ ]:
def test_image(url_or_path: str):
    """One-call image tester — works with the currently loaded model."""
    img = load_image(url_or_path)

    if has_custom and 'detector' in dir():
        # Custom model (e.g. AI image detector)
        r = detector.predict(url_or_path)
        show_image(img, f"{r['prediction']} ({r['confidence']:.1%})")
        return r

    elif processor is not None and task in ("image-classification",):
        # Standard HF image classifier
        inputs = processor(images=img, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            logits = model(**inputs).logits
        probs = torch.nn.functional.softmax(logits, dim=-1)[0]
        top = probs.topk(5)
        results = []
        for s, i in zip(top.values, top.indices):
            label = model.config.id2label.get(i.item(), f"class_{i.item()}")
            results.append({"Label": label, "Confidence": f"{s.item():.4f}"})
        show_image(img, results[0]["Label"])
        return pd.DataFrame(results)

    elif task:
        pipe = pipeline(task, model=MODEL_ID)
        r = pipe(img)
        show_image(img, str(r[0] if isinstance(r, list) else r))
        return r

def test_text(text: str):
    """One-call text tester — works with the currently loaded model."""
    if processor and task in ("text-generation", "text2text-generation"):
        inputs = processor(text, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            out = model.generate(**inputs, max_new_tokens=200, temperature=0.7, do_sample=True)
        return processor.decode(out[0], skip_special_tokens=True)
    elif processor and task in ("text-classification", "sentiment-analysis"):
        inputs = processor(text, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            logits = model(**inputs).logits
        probs = torch.nn.functional.softmax(logits, dim=-1)[0]
        results = {model.config.id2label.get(i, f"class_{i}"): f"{s:.4f}" for i, s in enumerate(probs)}
        return results
    elif task:
        pipe = pipeline(task, model=MODEL_ID)
        return pipe(text)

# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  PASTE YOUR OWN INPUT BELOW                                                ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

# test_image("https://example.com/your-image.jpg")
# test_image("/path/to/local/image.png")
# test_text("Your custom text here")

## 10. Swap to a Different Model

To try a completely different model, just change `MODEL_ID` and re-run from **Section 5** onwards.

In [ ]:
# ── Quick swap: uncomment one and re-run from Section 5 ──────────────────────

# MODEL_ID = "google/vit-base-patch16-224"                        # Image classification
# MODEL_ID = "microsoft/resnet-50"                                 # Image classification
# MODEL_ID = "facebook/detr-resnet-50"                             # Object detection
# MODEL_ID = "distilbert-base-uncased-finetuned-sst-2-english"    # Sentiment analysis
# MODEL_ID = "google/gemma-2-2b-it"                               # Text generation (needs token)
# MODEL_ID = "microsoft/Phi-3-mini-4k-instruct"                   # Text generation
# MODEL_ID = "openai/whisper-base"                                 # Speech-to-text
# MODEL_ID = "facebook/bart-large-cnn"                             # Summarisation
# MODEL_ID = "sentence-transformers/all-MiniLM-L6-v2"             # Embeddings
# MODEL_ID = "Salesforce/blip-image-captioning-base"              # Image captioning
# MODEL_ID = "stabilityai/stable-diffusion-xl-base-1.0"           # Image generation

# model, processor, info = load_any_model(MODEL_ID)